# Automated Report Production in Python

### Mindset: Setting up a factory. Slow for the first unit, vastly faster for all others.
* Real world example: 1 month to produce reports by hand; 1.5 days to send emails
* Now takes 12 minutes

## 1. Import packages
* Matplotlib: Create charts
* PIL: Image processing (use to add FIPs like the Canada logo)

In [ ]:
%matplotlib agg

import datetime
import math
from matplotlib import rcParams
import matplotlib.pyplot as plt
from PIL import Image

## 2. Build layout

### 2.1 Create figure
* Figure represents our page
* Set to legal-sized page, landscape orientation 

In [ ]:
fig = plt.figure()
fig.set_figheight(12.75)
fig.set_figwidth(21)
fig.set_facecolor('white')
rcParams['font.family'] = 'Calibri'

### 2.2 Add axes
* Figure is divided into axes
* Each axis is where you can place a chart or other content
* Add padding between the axes so titles don't overlap

In [ ]:
header =  plt.subplot2grid((18, 8), (0, 0), rowspan=1, colspan=8)
chart_1 = plt.subplot2grid((18, 8), (1, 0), rowspan=8, colspan=4)
chart_2 = plt.subplot2grid((18, 8), (1, 4), rowspan=8, colspan=4)
chart_3 = plt.subplot2grid((18, 8), (9, 0), rowspan=8, colspan=4)
chart_4 = plt.subplot2grid((18, 8), (9, 4), rowspan=8, colspan=4)
footer =  plt.subplot2grid((18, 8), (17, 0), rowspan=1, colspan=8)

fig.tight_layout(pad=1.0, h_pad=1.8)

In [ ]:
display(fig)

### 2.3 Add FIPs
* Use a helper function 'get_pic' to resize the FIPs to fit nicely on the page

In [ ]:
def get_pic(file, resize_factor=1):
    pic = Image.open(file)
    width  = math.floor(pic.size[0] / resize_factor)
    height = math.floor(pic.size[1] / resize_factor)
    pic = pic.resize((width, height), resample=Image.ANTIALIAS)
    return pic, width, height

# Top left
csps, width, height = get_pic('FIPs/csps.png', resize_factor=2.5)
fig.figimage(csps, xo=0, yo=fig.bbox.ymax - height, origin='upper', zorder=1)

# Bottom left
my_school, width, height = get_pic('FIPs/my_school.png', resize_factor=1.2)
fig.figimage(my_school, xo=0, yo=0, origin='upper', zorder=1)

# Bottom right
canada, width, height = get_pic('FIPs/canada.png', resize_factor=2.5)
fig.figimage(canada, xo=fig.bbox.xmax - width, yo=0, origin='upper', zorder=1)

In [ ]:
display(fig)

### 2.4 Add text

In [ ]:
footer.text(x=0.9, y=0.5, horizontalalignment='right', fontsize=8, s='See Appendix 1 for Methodology – ' + \
    'Consulter l\'Annexe 1 pour la méthodologie')

footer.text(x=0.9, y=-0.4, horizontalalignment='right', fontsize=8, s='Report generated on – ' + \
    'Rapport généré le ' + datetime.datetime.today().strftime('%d/%m/%Y'))

footer.text(x=0.9, y=-1.3, horizontalalignment='right', fontsize=8, s='Page 1/1')

In [ ]:
display(fig)

### 2.5 Remove unnecessary gridlines

In [ ]:
def remove_gridlines(ax):
    ax.axis('off')

remove_gridlines(header)
remove_gridlines(footer)

In [ ]:
display(fig)

## 3. The fun part: Add charts

### 3.1 Add a simple piechart to see how Matplotlib is used
* Can use hex to specify colours
* Your department likely has an official colour swatch with values in hex and RGB

In [ ]:
labels = ['Moose', 'Beaver', 'Chipmunk']
values = [4, 5, 6]
colour_list = ['#005261', '#A4BCC4', '#54575A']

chart_1.pie(x=values, labels=labels, explode=[0, 0.05, 0], colors=colour_list)
chart_1.set_title('Species Employed in NCR', fontsize=16)
# Fix skew
chart_1.axis('equal')

In [ ]:
display(fig)

### 3.2 Import a data set
* In practice, never want to manually enter numbers for charts, especially when automating
* Using the Pandas library, short for PANel DAta
* Tip: Ensure encodings match to prevent Français -> FranÃ§ais

In [ ]:
import pandas as pd

In [ ]:
store_data = pd.read_csv('store_data.csv', sep=',', index_col=0, encoding='utf-8')

In [ ]:
# For the database enthusiasts, we're importing a denormalized table
# Can import multiple files and perform joins in Pandas if needed
store_data.head()

### 3.3 Extract an interesting insight from the dataset
* Let's begin with coffee and calculate its monthly sales

In [ ]:
# Get rows where product_name is 'Coffee', and we're only interested in columns 'quantity', 'month', and 'month_num'
monthly_coffee = store_data.loc[store_data['product_name'] == 'Coffee', ['quantity', 'month', 'month_num']]
# Roll-up sales by month
monthly_coffee = monthly_coffee.groupby(['month', 'month_num'], as_index=False).sum()
# Sort
monthly_coffee = monthly_coffee.sort_values('month_num')

In [ ]:
monthly_coffee

### 3.4 Plot the insight on a chart

In [ ]:
chart_2.plot(monthly_coffee['month'], monthly_coffee['quantity'], color=colour_list[0])
chart_2.set_title('Monthly Coffee Sales', fontsize=16)

In [ ]:
display(fig)

## 4. Add remaining two charts
* Will skip explanation for sake of time but can use code as reference

### 4.1 Client satisfaction

In [ ]:
satisfied = store_data.loc[store_data['product_name'] == 'Coffee', ['satisfied']].sum()
total = store_data.loc[store_data['product_name'] == 'Coffee', ['satisfied']].count()
dissatisfied = total - satisfied
colour_list = ['#005261', '#567582']

chart_3.bar(x=['yo', 'pce'], height=[int(satisfied), int(dissatisfied)])
chart_3.set_title('Customer Satisfaction by Product', fontsize=16)

In [ ]:
# Show percentages on y-axis
chart_3.set_ylim(0, 1.1)
vals = chart_3.get_yticks()
chart_3.set_yticklabels(['{:,.0%}'.format(x) for x in vals])

# Plot data
bars = 


# Set colours for bars
bars[0].set_color(colour_list[0])
bars[1].set_color(colour_list[1])
bars[2].set_color(colour_list[2])
bars[3].set_color(colour_list[3])

### 4.2 Market share

In [ ]:
coffee_share = store_data['product_name'].value_counts()['Coffee']
total_market = store_data['product_name'].value_counts().sum()
other_products = total_market - coffee_share
colour_list = ['#005261', '#A4BCC4']

chart_4.pie(x=[coffee_share, other_products], labels=['Coffee', 'Other Products'], colors=colour_list)
chart_4.set_title('Coffee Market Share', fontsize=16)
chart_4.axis('equal')

In [ ]:
display(fig)

## 5. Export to PDF

In [ ]:
fig.savefig('DM Report - Rapport du SM.pdf')

In [ ]:
# Add for loop, show audience can extend this to generate many, many reports at once